In [81]:
# Imports
import sqlite3
import pandas as pd
import re
import os

In [82]:
# Connecting the database
DATABASE_PATH = os.path.join('..', '..', 'dist', 'database.sqlite')
con = sqlite3.connect(DATABASE_PATH)
SCRAPE_ID=1

In [83]:
# Reading the database
df = pd.read_sql_query(f"""SELECT
  id,
  job_url,
  job_title,
  company_name,
  company_description,
  job_description,
  preferred_experience,
  recruitement_process
  FROM job_offers WHERE scrape_id={SCRAPE_ID}""", con)

In [84]:
df = df.set_index('id')

In [85]:
df = df.dropna(subset=['job_url', 'job_title'])

In [86]:
# Trimming the job url parameters
df['job_url'] = df['job_url'].str.split('?').str[0]

In [87]:
df = df.drop_duplicates(subset='job_url')

In [88]:
regex = {
    'Excel': r'\bExcel\b',
    'VBA': r'\bVBA\b',
    'R': r'\bR($|[\s\.\,])',
    'Python': r'\bPython\b',
    'SQL': r'\b\w*SQL\w*\b',
    'Power BI': r'\bPower\s?BI\b',
    'Tableau': r'\bTableau\b',
    'Looker': r'\bLooker\b'
}

def contains_word(text: str, word: str) -> bool:
    """ Checks if a word in contained in a text"""
    if text is not None and isinstance(text, str):
        pattern = re.compile(regex[word], flags=re.IGNORECASE)
        return bool(pattern.search(text))
    return False

In [89]:
# TODO: use fuzzy string search to match misspelled words and get better accuracy
words_to_check = ['Excel', 'VBA', 'R', 'Python', 'SQL', 'Power BI', 'Tableau', 'Looker']
result_df = pd.DataFrame(columns=['job_offer_id'] + words_to_check)
result_df = result_df.set_index('job_offer_id')

In [90]:
columns = ['job_description', 'preferred_experience', 'recruitement_process'] # columns to check the keywords in

for index, row in df[columns].iterrows():
    result_row = {}
    for word in words_to_check:
        result_row[word] = int(contains_word(row['job_description'], word) or contains_word(row['preferred_experience'], word) or contains_word(row['recruitement_process'], word))
    result_df.loc[index] = result_row
    result_df


In [91]:
# Drop all the rows where all the columns are equal to 0
result_df = result_df.drop(result_df[(result_df == 0).all(axis=1)].index)

In [93]:
# Export
SAVE_DIR=os.path.join('..', '..', 'dist')
SAVE_PATH_EXCEL=os.path.join(SAVE_DIR, 'output.xlsx')

result_df.to_excel(SAVE_PATH_EXCEL)